## Specifying Batch Size While Training A Model

In [1]:
from torch.utils.data import Dataset,DataLoader
import torch
import torch.nn as nn

In [2]:
x=[[1,2],[3,4],[5,6],[7,8]]
y=[[3],[7],[11],[15]]

In [3]:
X=torch.tensor(x).float()
Y=torch.tensor(y).float()

In [4]:
device='cuda' if torch.cuda.is_available() else 'cpu'
X=X.to(device)
Y=Y.to(device)

In [5]:
class MyDataset(Dataset):
    def __init__(self,x,y):
        self.x=torch.tensor(x).float()
        self.y=torch.tensor(y).float()
    def __len__(self):
        return len(self.x)
    def __getitem__(self, ix):
        return self.x[ix],self.y[ix]

In [6]:
ds=MyDataset(X,Y)

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_16104\856202636.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.x=torch.tensor(x).float()
C:\Users\Lenovo\AppData\Local\Temp\ipykernel_16104\856202636.py:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.y=torch.tensor(y).float()


In [7]:
dl=DataLoader(ds,batch_size=2,shuffle=True)

In [8]:
# NOTE - This line of code is not a part of model building,
# this is used only for illustration of 
# how to print the input and output batches of data
for x,y in dl:
    print(x,y)

tensor([[1., 2.],
        [7., 8.]], device='cuda:0') tensor([[ 3.],
        [15.]], device='cuda:0')
tensor([[3., 4.],
        [5., 6.]], device='cuda:0') tensor([[ 7.],
        [11.]], device='cuda:0')


In [9]:
class MyNeuralNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.input_to_hidden_layer=nn.Linear(2,8)
        self.hidden_layer_activation=nn.ReLU()
        self.hidden_to_output_layer=nn.Linear(8,1)
    def forward(self,x):
        x=self.input_to_hidden_layer(x)
        x=self.hidden_layer_activation(x)
        x=self.hidden_to_output_layer(x)
        return x

In [10]:
mynet=MyNeuralNet().to(device)
loss_func=nn.MSELoss()
from torch.optim import SGD
opt=SGD(mynet.parameters(),lr=0.001)

In [11]:
import time
loss_history=[]
start=time.time()
for _ in range(50):
    for data in dl:
        x,y=data
        opt.zero_grad()
        loss_value=loss_func(mynet(x),y)
        loss_value.backward()
        opt.step()
        loss_history.append(loss_value)
end=time.time()
print(end-start)

0.13586759567260742


In [12]:
val_x=[[10,11]]

In [13]:
val_x=torch.tensor(val_x).float().to(device)

In [14]:
mynet(val_x)

tensor([[20.5776]], device='cuda:0', grad_fn=<AddmmBackward0>)